In [ ]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf
print("TF: ", tf.__version__)
from tensorflow.keras import layers
from tensorflow.keras import models


import wandb
print("W&B: ", wandb.__version__)
from wandb.keras import WandbCallback

import medmnist
print("medMNIST: ", medmnist.__version__)
from medmnist import INFO



: 

In [2]:
wandb.login()

wandb: Currently logged in as: beomgiso. Use `wandb login --relogin` to force relogin


True

In [3]:
configs = dict(
    data_flag = 'bloodmnist',
    image_width = 32,
    image_height = 32,
    batch_size = 128,
    model_name = 'vgg16',
    pretrain_weights = 'imagenet',
    epochs = 100,
    init_learning_rate = 0.001,
    lr_decay_rate = 0.1,
    optimizer = 'adam',
    loss_fn = 'sparse_categorical_crossentropy',
    metrics = ['acc'],
    earlystopping_patience = 5
)



In [4]:
info = INFO[configs['data_flag']]
configs['class_names'] = info['label']
configs['image_channels'] = info['n_channels']

info

{'python_class': 'BloodMNIST',
 'description': 'The BloodMNIST is based on a dataset of individual normal cells, captured from individuals without infection, hematologic or oncologic disease and free of any pharmacologic treatment at the moment of blood collection. It contains a total of 17,092 images and is organized into 8 classes. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images with resolution 3×360×363 pixels are center-cropped into 3×200×200, and then resized into 3×28×28.',
 'url': 'https://zenodo.org/record/6496656/files/bloodmnist.npz?download=1',
 'MD5': '7053d0359d879ad8a5505303e11de1dc',
 'task': 'multi-class',
 'label': {'0': 'basophil',
  '1': 'eosinophil',
  '2': 'erythroblast',
  '3': 'immature granulocytes(myelocytes, metamyelocytes and promyelocytes)',
  '4': 'lymphocyte',
  '5': 'monocyte',
  '6': 'neutrophil',
  '7': 'platelet'},
 'n_channels': 3,
 'n_samples': {'train': 11959, 'val': 1712, 'test': 3421},
 '

In [5]:
#@title
def download_and_prepare_dataset(data_info: dict):
    """
    Utility function to download the dataset and return train/valid/test images/labels.

    Arguments:
        data_info (dict): Dataset metadata
    """
    data_path = tf.keras.utils.get_file(origin=data_info['url'], md5_hash=data_info['MD5'])

    with np.load(data_path) as data:
        # Get images
        train_images = data['train_images']
        valid_images = data['val_images']
        test_images = data['test_images']

        # Get labels
        train_labels = data['train_labels'].flatten()
        valid_labels = data['val_labels'].flatten()
        test_labels = data['test_labels'].flatten()

    return train_images, train_labels, valid_images, valid_labels, test_images, test_labels

In [6]:
train_images, train_labels, valid_images, valid_labels, test_images, test_labels = download_and_prepare_dataset(info)

In [7]:
# For demonstration purposes
log_full = False #@param {type:"boolean"}

if log_full:
    log_train_samples = len(train_images)
else:
    log_train_samples = 1000 
    

print(f'Number of train images : {log_train_samples} to be logged')

Number of train images : 1000 to be logged


In [8]:
%%time

# Initialize a new W&B run
run = wandb.init(project='medmnist-bloodmnist', group='viz_data')

# Intialize a W&B Artifacts
ds = wandb.Artifact("medmnist_bloodmnist_dataset", "dataset")

# Initialize an empty table
train_table = wandb.Table(columns=[], data=[])
# Add training data
train_table.add_column('image', train_images[:log_train_samples])
# Add training label_id
train_table.add_column('label_id', train_labels[:log_train_samples])
# Add training class names
train_table.add_computed_columns(lambda ndx, row:{
    "images": wandb.Image(row["image"]),
    "class_names": configs['class_names'][str(row["label_id"])]
    })

# Add the table to the Artifact
ds['train_data'] = train_table

# Let's do the same for the validation data
valid_table = wandb.Table(columns=[], data=[])
valid_table.add_column('image', valid_images)
valid_table.add_column('label_id', valid_labels)
valid_table.add_computed_columns(lambda ndx, row:{
    "images": wandb.Image(row["image"]),
    "class_name": configs['class_names'][str(row["label_id"])]
    })
ds['valid_data'] = valid_table

# Save the dataset as an Artifact
ds.save()

# Finish the run
wandb.finish()

CPU times: user 791 ms, sys: 335 ms, total: 1.13 s
Wall time: 9.71 s


In [9]:
#@title
@tf.function
def preprocess(image: tf.Tensor, label: tf.Tensor):
    """
    Preprocess the image tensors and parse the labels
    """
    # Preprocess images
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    # Parse label
    label = tf.cast(label, tf.float32)
    
    return image, label


def prepare_dataloader(images: np.ndarray,
                       labels: np.ndarray,
                       loader_type: str='train',
                       batch_size: int=128):
    """
    Utility function to prepare dataloader.
    """
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))

    if loader_type=='train':
        dataset = dataset.shuffle(1024)

    dataloader = (
        dataset
        .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    return dataloader

In [10]:
trainloader = prepare_dataloader(train_images, train_labels, 'train', configs.get('batch_size', 64))
validloader = prepare_dataloader(valid_images, valid_labels, 'valid', configs.get('batch_size', 64))
testloader = prepare_dataloader(test_images, test_labels, 'test', configs.get('batch_size', 64))

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-06-14 20:55:20.233261: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-14 20:55:20.233399: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
img_augmentation = models.Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomFlip()],
    name="img_augmentation",
)

In [12]:
#@title
def augment_5_times(img):
    augmented_imgs = []
    for _ in range(5):
        aug_img = tf.squeeze(img_augmentation(img), axis=0)
        # Notice the use of wrapping the images with wandb.Image
        wandb_image = wandb.Image(aug_img.numpy())
        augmented_imgs.append(wandb_image)

    return augmented_imgs

In [13]:
%%time

viz_augment_samples = 100

# Initialize a W&B run
run = wandb.init(project='medmnist-bloodmnist', group='viz_augmentation')

# Use the already logged dataset
train_art = run.use_artifact('ayush-thakur/medmnist-bloodmnist/medmnist_bloodmnist_dataset:latest', type='dataset')

# Get the train_table to access the data
train_table = train_art.get("train_data")

# Get the images, ground truth label, and row index
images = train_table.get_column("images", convert_to="numpy")
labels = train_table.get_column("label_id", convert_to="numpy")
ids = train_table.get_index()
# Shuffle the ids and slice
random.shuffle(ids)
sample_ids = ids[0:viz_augment_samples]

# Create augmentation table
augment_table = wandb.Table(columns=['image', 'truth', 'label_id', 'aug1', 'aug2', 'aug3', 'aug4', 'aug5'])

# Get augmented images and log it onto the table
for sample_id in sample_ids:
    img = images[sample_id]
    label = labels[sample_id]
    augmented_imgs = augment_5_times(tf.expand_dims(img, axis=0))
    augment_table.add_data(wandb.Image(img),
                           np.argmax(label),
                           configs['class_names'][str(label)],
                           augmented_imgs[0],
                           augmented_imgs[1],
                           augmented_imgs[2],
                           augmented_imgs[3],
                           augmented_imgs[4])

# Log the table
wandb.log({'augmented data': augment_table})

# Finish the run
wandb.finish()

NotFoundError: Exception encountered when calling layer "random_rotation" (type RandomRotation).

No registered 'RngReadAndSkip' OpKernel for 'GPU' devices compatible with node {{node RngReadAndSkip}}
	.  Registered:  device='CPU'
 [Op:RngReadAndSkip]

Call arguments received:
  • inputs=tf.Tensor(shape=(1, 28, 28, 3), dtype=int64)
  • training=True

In [14]:
#@title
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def create_gradcam(image, model, last_conv_layer_name, pred_index=None):
    # Preprocess the image array
    image, _ = preprocess(tf.expand_dims(image, axis=0), 0)
    # Get GradCAM
    heatmap = make_gradcam_heatmap(image, model, last_conv_layer_name, pred_index)
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]
    jet_heatmap = tf.image.resize(jet_heatmap, size=(28,28))

    # Overlay
    superimposed_img = jet_heatmap * 0.4 + tf.squeeze(image, axis=0)
    superimposed_img = tf.clip_by_value(superimposed_img, 0.0, 1.0)

    return superimposed_img

In [16]:
last_conv_layer_name = 'block4_conv3'

In [15]:
def validation_processor(ndx, row):
    return {
        "input:image": wandb.Image(row["input"]),
        "target:class": class_table.index_ref(row["target"])
    }

def prediction_processor(ndx, row):
    # Get the validation image
    valid_image = np.array(row["val_row"].get_row()["input:image"].image)

    return {
        "output:class": class_table.index_ref(np.argmax(row["output"])),
        "gradcam": wandb.Image(create_gradcam(valid_image, model, last_conv_layer_name)),
        "output:logits": {class_name: value for (class_name, value) in zip(list(config.class_names.values()), row["output"].tolist())}
    }

In [18]:
def get_model(input_shape: tuple=(28, 28, 3), 
              resize: tuple=(32, 32, 3),
              dropout_rate: float=0.5,
              num_classes: int=8,
              output_activation: str='softmax'):
  
    inputs = layers.Input(input_shape)
    resize_img = layers.Resizing(resize[0], resize[1], interpolation='bilinear')(inputs)
    augment_img = img_augmentation(resize_img)
  
    base_model = tf.keras.applications.VGG16(include_top=False, 
                                             weights=configs['pretrain_weights'], 
                                             input_shape=resize,
                                             input_tensor=augment_img)
    base_model.trainabe = True

    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation=output_activation)(x)

    return models.Model(inputs, outputs)

tf.keras.backend.clear_session()
model = get_model()
model.summary()

58900480/58889256 [==============================] - 6s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 3)]       0         
                                                                 
 resizing (Resizing)         (None, 32, 32, 3)         0         
                                                                 
 img_augmentation (Sequentia  (None, None, None, 3)    0         
 l)                                                              
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0      

In [20]:
earlystopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=configs['earlystopping_patience'], verbose=0, mode='auto',
    restore_best_weights=True
)

In [21]:
def lr_scheduler(epoch, lr):
    # log the current learning rate onto W&B
    if wandb.run is None:
        raise wandb.Error("You must call wandb.init() before WandbCallback()")

    wandb.log({'learning_rate': lr}, commit=False)
    
    if epoch < 7:
        return lr
    else:
        return lr * tf.math.exp(-configs['lr_decay_rate'])

lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [22]:
def train(config: dict, 
          callbacks: list,
          verbose: int=0):
    """
    Utility function to train the model.

    Arguments:
        config (dict): Dictionary of hyperparameters.
        callbacks (list): List of callbacks passed to `model.fit`.
        verbose (int): 0 for silent and 1 for progress bar.
    """

    # Initalize model
    tf.keras.backend.clear_session()
    model = get_model(resize=(config.image_width, config.image_height, config.image_channels))

    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=config.init_learning_rate)
    model.compile(opt,
                  config.loss_fn,
                  metrics=config.metrics)

    # Train the model
    _ = model.fit(trainloader,
                  epochs=config.epochs,
                  validation_data=validloader,
                  callbacks=callbacks,
                  verbose=verbose)

    return model

In [23]:
# Initialize the W&B run
run = wandb.init(project='medmnist-bloodmnist', config=configs, job_type='train')
config = wandb.config

# Get validation table
data_art = run.use_artifact('ayush-thakur/medmnist-bloodmnist/medmnist_bloodmnist_dataset:latest', type='dataset')
valid_table = data_art.get("valid_data")

# Create a class table
class_table = wandb.Table(columns=[], data=[])
class_table.add_column("class_name", list(config.class_names.values()))

# Define WandbCallback for experiment tracking
wandb_callback = WandbCallback(
                    log_evaluation=True,
                    validation_row_processor=lambda ndx, row: validation_processor(ndx, row),
                    prediction_row_processor=lambda ndx, row: prediction_processor(ndx, row),
                    validation_steps=4,
                    save_model=False
                )

# callbacks
callbacks = [earlystopper, wandb_callback, lr_callback]

# Train
model = train(config, callbacks=callbacks, verbose=1)

# Evaluate the trained model
loss, acc = model.evaluate(validloader)
wandb.log({'evaluate/accuracy': acc})

# Close the W&B run.
wandb.finish()

2022-06-14 20:58:22.518071: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


InvalidArgumentError: Cannot assign a device for operation model/img_augmentation/random_rotation/stateful_uniform/RngReadAndSkip: Could not satisfy explicit device specification '' because the node {{colocation_node model/img_augmentation/random_rotation/stateful_uniform/RngReadAndSkip}} was colocated with a group of nodes that required incompatible device '/job:localhost/replica:0/task:0/device:GPU:0'. All available devices [/job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:GPU:0]. 
Colocation Debug Info:
Colocation group had the following types and supported devices: 
Root Member(assigned_device_name_index_=2 requested_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' assigned_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' resource_device_name_='/job:localhost/replica:0/task:0/device:GPU:0' supported_device_types_=[CPU] possible_devices_=[]
RngReadAndSkip: CPU 
_Arg: GPU CPU 

Colocation members, user-requested devices, and f